In [1]:
import pandas as pd
import numpy as np
import tables
import datetime as dt


exchange = 'NSE'
symbol = "HDFC"
today = '2019-04-05'#dt.datetime.now().strftime("%d_%m_%y")
prevday = '2019-04-04'
sample_freq = '5min'

# OAuth Login

In [2]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

#logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="b2w0sfnr1zr92nxm")
#kite.login_url()

if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
else:
    driver = webdriver.Chrome("./automation/chromedriver")

driver.get(kite.login_url())

In [3]:
# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.
request_token = re.findall("request_token=(.*)&action",driver.current_url)

data = kite.generate_session(request_token[0], api_secret="jtga2mp2e5fn29h8w0pe2kb722g3dh1q")
kite.set_access_token(data["access_token"])

driver.close()

# Stock List Download

In [4]:
def getInstruments(exchange='NSE'):
    instruments_df = pd.DataFrame(data=kite.instruments(exchange))
    instruments_df = instruments_df.set_index('tradingsymbol')
    return instruments_df

In [6]:
instruments_df = getInstruments(exchange)
instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange+today,mode='w')

# Historical Data Download

In [7]:
def downloadData(symbol="HDFC", fromDate= dt.datetime.now() - dt.timedelta(days = 1), toDate=dt.datetime.now(), freq="minute"):
    symbolToken = instruments_df.loc[symbol,'instrument_token']
    raw_data = pd.DataFrame(data=kite.historical_data(symbolToken, fromDate, toDate, freq, continuous=False))
    raw_data = raw_data.set_index('date').tz_localize(None)
    return raw_data
                

In [8]:
def resample(data,freq):
    data = data.resample(freq).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'})
    #data.columns = data.columns.droplevel()
    return data
    
    

In [54]:
raw_data = downloadData(symbol, dt.datetime.now() - dt.timedelta(days = 3), dt.datetime.now() - dt.timedelta(days = 1))
raw_data_day = downloadData(symbol, dt.datetime.now() - dt.timedelta(days = 2*365),dt.datetime.now(), freq="day")


raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+today+"day",mode='a')
raw_data.to_hdf('kite_data/kite_data.h5', key=symbol+today,mode='a')

In [3]:
raw_data = pd.read_hdf('kite_data/kite_data.h5', key=symbol+today,mode='r')
raw_data_day = pd.read_hdf('kite_data/kite_data.h5', key=symbol+today+"day",mode='r')

# Filter Stocks

# Live Data resampling

# Indicators calculation

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50

yMin = raw_data.iloc[-1*noofcandles:-1]['low'].min()-10
yMax = raw_data.iloc[-1*noofcandles:-1]['high'].max()
xMin = raw_data.index[-1*noofcandles]
xMax = raw_data.index[-1]


########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

#################################
#   Plotting Data              #
###############################
init_notebook_mode(connected=True)

In [33]:
from talib import MACD
from talib import RSI, BBANDS, MACD, AROON, STOCH, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC
from talib import CDLABANDONEDBABY, CDL3BLACKCROWS,CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLMARUBOZU,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLSPINNINGTOP,CDL3BLACKCROWS, CDL3LINESTRIKE


def candlestick(price, pos=1, plot=False):
    # Candlestick
    trace = go.Candlestick(x=price.index, open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Candlestick", showlegend=False)

    if plot:
        fig.append_trace(trace, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="Candlestick"
    return price

def macd(price, pos=1, plot=False):
    price['macd'], price['macdsignal'], price['macdhist'] = MACD(price.close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    # MACD plots
    traceMACD = go.Scatter(x=price.index, y=price.macd, name='MACD', line=dict(color='black'),showlegend=False)
    traceMACDSignal = go.Scatter(x=price.index, y=price.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
    traceMACDHist = go.Bar(x=price.index, y=price.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        
    if plot:
        fig.append_trace(traceMACD, pos, 1)
        fig.append_trace(traceMACDSignal, pos, 1)
        fig.append_trace(traceMACDHist, pos, 1)
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="MACD"
    
    return price

def rsi(price, pos=1, plot=False):
    price['RSI'] = RSI(price.close, timeperiod=14)
    
    traceRSI = go.Scatter(x=price.index, y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI',showlegend=False)
    
    if plot:
        fig.append_trace(traceRSI, pos, 1)
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
        fig['layout']['yaxis'+str(pos)]['tickvals']=[20,30,70,80]
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="RSI"
        
    return price

def stoch(price, pos=1, plot=False):
    price['slowk'], price['slowd'] = STOCH(price.high, price.low, price.close)
    
    traceSK  = go.Scatter(x=price.index, y=price['slowk'], name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
    traceSD  = go.Scatter(x=price.index, y=price['slowd'], name='%D', line=dict(color='red'),showlegend=False)
    
    if plot:
        fig.append_trace(traceSK, pos, 1)
        fig.append_trace(traceSD, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="Stochastics"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['anchir']="x"
        fig['layout']['yaxis'+str(pos)]['tickvals']=[20,80,30,70]
    return price

def aroon(price, pos=1, plot=False):
    price['aroondown'], price['aroonup'] = AROON(price.high, price.low)
    
    traceARD  = go.Scatter(x=price.index, y=price['aroondown'], name='AROON Down', line=dict(color='red'), yaxis='y4',showlegend=False)
    traceARU  = go.Scatter(x=price.index, y=price['aroonup'], name='AROON UP', line=dict(color='green'),showlegend=False)
    
    if plot:
        fig.append_trace(traceARD, pos, 1)
        fig.append_trace(traceARU, pos, 1)
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="Aroon"
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
    return price

def atr(price, pos=1, plot=False, multiplier=2):
    price['atr'] = ATR(price.high, price.low, price.close, timeperiod=22)
    price['chandlierLong'] = price.close-multiplier*price.atr
    price['chandlierShort'] = price.close+multiplier*price.atr
    
    traceATR = go.Scatter(x=price.index, y=price['atr'],mode='lines', line=dict(color='lightgrey'), name='ATR',showlegend=False)
    
    traceStopLoss = go.Scatter(x=price.index, y=price['chandlierLong'].shift(1), mode='lines', line=dict(color='lightgrey'), name='SL',showlegend=False)
    if plot:
        fig.append_trace(traceStopLoss, pos, 1)
        
    return price

def bbands(price, pos=1, plot=False, plotPrice=False):
    price['bbup'], price['bbmid'], price['bblow'] = BBANDS(price.close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    
    tracePrice = go.Scatter(x=price.index, y=price.close, marker = dict(color='grey', size=2), mode='lines', name="Close Price", yaxis='y1', showlegend=False)
    traceBBU = go.Scatter(x=price.index, y=price['bbup'], name='BB_up',  line=dict(color='grey'),showlegend=False)
    traceBBL = go.Scatter(x=price.index, y=price['bblow'], name='BB_low',  line=dict(color='grey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    traceBBM = go.Scatter(x=price.index, y=price.bbmid, name='BB_mid',  line=dict(color='grey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
    
    
    if plot:
        if plotPrice:
            fig.append_trace(tracePrice, pos, 1)
            
        fig.append_trace(traceBBU, pos, 1)
        fig.append_trace(traceBBL, pos, 1)
        fig.append_trace(traceBBM, pos, 1)
    
    return price

def obv(price, pos=1, plot=False):
    price['obv'] = OBV(np.linspace(1,100,price.index.shape[0]), price.volume)
    
    traceV = go.Bar(x=price.index, y=price.volume, name='Volume', marker=dict(color='blue'), showlegend=False)
    traceOBV = go.Scatter(x=price.index, y=price['obv'], name='OBV', line=dict(color='black'),showlegend=False)
    if plot:
        fig.append_trace(traceV, pos, 1)
        
        #fig['layout']['yaxis2']['overlaying']='y'
        fig['layout']['yaxis'+str(pos)]['anchor']='x'
        fig['layout']['yaxis'+str(pos)]['showgrid']=True
        fig['layout']['yaxis'+str(pos)]['visible']=True
        
        maxV = round(raw_data.iloc[-3*noofcandles:-1]['volume'].max()/2,0)*2
        fig['layout']['yaxis'+str(pos)]['tickvals']=[maxV/2,maxV]
        fig['layout']['yaxis'+str(pos)]['range']=[0,maxV*2]
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['title']="volume"
    return price

def adosc(price, pos=1, plot=False):
    
    price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)
    return price

def adx(price, pos=1, plot=False):
    price['ADX'] = ADX(price.high, price.low, price.close)
    price['DI_P'] = PLUS_DI(price.high, price.low, price.close)
    price['DI_M'] = MINUS_DI(price.high, price.low, price.close)
    
    traceADX  = go.Scatter(x=price.index, y=price['ADX'], name='ADX', line=dict(color='blue') ,showlegend=False)
    traceDIP  = go.Scatter(x=price.index, y=price['DI_P'], name='DI+', line=dict(color='green'),showlegend=False)
    traceDIM  = go.Scatter(x=price.index, y=price['DI_M'], name='DI-', line=dict(color='red'),showlegend=False)
    
    
    if plot:
        fig.append_trace(traceADX, pos, 1)
        fig.append_trace(traceDIP, pos, 1)
        fig.append_trace(traceDIM, pos, 1)
        fig['layout']['yaxis'+str(pos)]['title']="ADX"
        fig['layout']['yaxis'+str(pos)]['side']="right"
        fig['layout']['yaxis'+str(pos)]['anchor']="x"
    return price

# Heikin-Ashi
def ha(price, pos=1, plot=False):
    price['HAopen'] = (price.open.shift(1) + price.close.shift(1))/2
    price['HAhigh'] = price[['high','open','close']].max(axis = 1, skipna = True)
    price['HAlow'] = price[['low','open','close']].min(axis = 1, skipna = True)
    price['HAclose'] = (price.open+price.high+price.low+price.close)/4
    
    traceHA = go.Candlestick(x=price.index, open=price['HAopen'], high=price['HAhigh'], low=price['HAlow'], close=price['HAclose'], name="Heikin Ashi", showlegend=False)
    
    if plot:
        fig.append_trace(traceHA, pos, 1)
    return price

def pivotPoint(price, pos=1,fibo=True, noofcandles=60, plot=False):
    prev = raw_data_day.loc[prevday]

    PP = (prev.high+prev.low+prev.close)/3
    S1 = PP - 0.382 * (prev.high - prev.low)
    S2 = PP - 0.618 * (prev.high - prev.low)
    S3 = PP -         (prev.high - prev.low)

    R1 = PP + 0.382 * (prev.high - prev.low)
    R2 = PP + 0.618 * (prev.high - prev.low)
    R3 = PP +         (prev.high - prev.low)
    
    tracePP  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],PP), line=dict(color='red'),showlegend=False)
    traceS1  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],S1), line=dict(color='red'),showlegend=False)
    traceS2  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],S2), line=dict(color='red'),showlegend=False)
    traceS3  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],S3), line=dict(color='red'),showlegend=False)
    traceR1  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],R1), line=dict(color='red'),showlegend=False)
    traceR2  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],R2), line=dict(color='red'),showlegend=False)
    traceR3  = go.Scatter(x=price.index, y=np.full(raw_data.index.shape[0],R3), line=dict(color='red'),showlegend=False)
    
    
    
    if plot:
        fig.append_trace(tracePP, pos, 1)
        fig.append_trace(traceS1, pos, 1)
        fig.append_trace(traceS2, pos, 1)
        fig.append_trace(traceS3, pos, 1)
        fig.append_trace(traceR1, pos, 1)
        fig.append_trace(traceR2, pos, 1)
        fig.append_trace(traceR3, pos, 1)
    return price

def detectCDPattern(prices, plot=True, noofcandles=15, strPlot="engulfing"):
    #https://github.com/mrjbq7/ta-lib
    
    annotateText = []
    annotateIndex = []
    
    for index, curr in prices.iloc[noofcandles:].iterrows():
        text = ""
        
        index_r = raw_data.index.get_loc(index)+1
        index_l = max(0, index_r - noofcandles)
        #rice = pd.DataFrame()
        popen = prices.iloc[index_l:index_r]['open']
        phigh = prices.iloc[index_l:index_r]['high']
        pclose = prices.iloc[index_l:index_r]['close']
        plow = prices.iloc[index_l:index_r]['low']
        
        
        if (CDLDOJI(popen, phigh, plow, pclose)[-1] != 0) and (strPlot.find("Doji") >= 0):
            text += "Doji,"
        
        if CDLENGULFING(popen, phigh, plow, pclose)[-1] == -100 and strPlot.find("engulfing")>=0:
            text += "Bearish Engulfing,"
            
        if CDLENGULFING(popen, phigh, plow, pclose)[-1] == 100 and strPlot.find("engulfing")>=0:
            #print(CDLENGULFING(popen, phigh, plow, pclose))
            #print(index.strftime("%d-%m-%y %H:%M"))
            text += "Bullish Engulfing,"
            
        
        if CDLABANDONEDBABY(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("abandonedbaby")>=0:
            text += "Abandoned Baby,"
    
        if CDL3BLACKCROWS(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("3blackcrows")>=0:
            text += "3 Black Crows,"
    
    
        if CDLDOJISTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("dojistar")>=0:
            text += "Doji Star,"
    
        if CDLDRAGONFLYDOJI(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("dragonflydoji")>=0:
            text += "Dragon Fly Doji,"
    
        if CDLEVENINGDOJISTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("eveningdojistar")>=0:
            text += "Evening Doji Star,"
    
        if CDLEVENINGSTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("eveningstar")>=0:
            text += "Evening Star,"
    
        if CDLGRAVESTONEDOJI(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("gravestonedoji")>=0:
            text += "Gravestone Doji,"
    
        if CDLHAMMER(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("hammer")>=0:
            text += "Hammer,"
    
        if CDLHANGINGMAN(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("hangingman")>=0:
            text += "Hanging Man,"
    
        if CDLHARAMI(popen, phigh, plow, pclose)[-1] == 100 and strPlot.find("Harami")>=0:
            text += "Bullish Harami,"
            
        if CDLHARAMI(popen, phigh, plow, pclose)[-1] == -100 and strPlot.find("Harami")>=0:
            text += "Bearish Harami,"
    
        if CDLHARAMICROSS(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("haramicross")>=0:
            text += "Harami Cross,"
    
        if CDLINVERTEDHAMMER(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("invertedhammer")>=0:
            text += "Inverted Hammer,"
    
        if CDLMARUBOZU(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("marubozu")>=0:
            text += "Marubozu,"
    
        if CDLMORNINGDOJISTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("morningdojistar")>=0:
            text += "Morning Doji Star,"
    
        if CDLMORNINGSTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("morningstar")>=0:
            text += "Morning Star,"
    
        if CDLSHOOTINGSTAR(popen, phigh, plow, pclose)[-1] != 0 and strPlot.find("shootingstar")>=0:
            text += "Shooting Star,"
    
    
        if CDLSPINNINGTOP(popen, phigh, plow, pclose)[-1] and strPlot.find("spinningtop")>=0:
            text += "Spinning Top,"
    
        if CDL3LINESTRIKE(popen, phigh, plow, pclose)[-1] and strPlot.find("3linestrike")>=0:
            text += "3 Line Strike,"
        
        
        if text != "":
            annotateText.append(text)
            annotateIndex.append(index)
            
            #print(index.strftime("%d-%m-%y %H:%M")+": "+text)
    

    data = pd.DataFrame({'open':prices.loc[annotateIndex, 'open'],
                         'close':prices.loc[annotateIndex, 'close'],
                         'high':prices.loc[annotateIndex, 'high'],
                         'low':prices.loc[annotateIndex, 'low'],
                         'text':annotateText})
    prices['text'] =""
    
    prices['text'] = data['text']
    
    if plot:
        annotateBuySell(data)
    return prices

def crossover(price):
    return

def divergence(price):
    return


def calculateStats(price):
    price['priceSD'] = STDDEV(price.close)
    price['priceVAR'] = VAR(price.close)

    price['delBBUP'] = (price.bbup - price.close)/(price.bbup-price.bbmid)
    price['delBBLOW'] = (price.close - price.bblow)/(price.bbmid-price.bblow)

    price['priceTSF'] = TSF(price.close)

    price['priceROC'] = ROC(price.close)
    price['macdROC'] = ROC(price.macd)
    price['bbROC'] = ROC(price.bbmid)

    price['sdPriceRange'] = STDDEV((price.high - price.close))
    price['priceRange'] = (price.high - price.close)

    return price


def annotateBuySell(price, AnnotateType=""):
    arr = []
    for index, row in price.iterrows():
        if AnnotateType == "Buy":
            arr.append(dict(x=index,y=row['low']-1, xref='x',yref='y', ax=0, ay=35,
                                       showarrow=True, arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='green'))
        elif AnnotateType == "Sell":
            arr.append(dict(x=index,y=row['high']+1, xref='x',yref='y', ax=0, ay=-35,
                                       showarrow=True, arrowhead=1,arrowsize=0.5, arrowwidth=7,arrowcolor='red'))
        else:
            arr.append(dict(x=index,y=row['low']-1, xref='x',yref='y', ax=0, ay=50, text=row["text"],
                                       showarrow=True, arrowhead=1,arrowsize=1, arrowwidth=1,arrowcolor='black'))
            
    
    arr.extend(fig['layout']['annotations'])
    fig['layout']['annotations']=arr
    return price

def createPlot():
    fig = tools.make_subplots(rows=5, cols=1, shared_xaxes=True, row_width=[1,1,3,1,5], vertical_spacing = 0.01)
    fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom", range=[xMin,xMax])
    fig['layout'].update(height=950, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['anchor'] = 'x'
    fig['layout']['yaxis']['side'] = 'right'
    
    fig['layout']['xaxis']['rangeselector'] = dict(
                buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                              dict(count=3, label='3h', step='hour', stepmode='backward'),
                              dict(count=6, label='1d', step='hour', stepmode='backward'),
                              dict(step='all')]))
    return fig


fig = createPlot()
raw_data = candlestick(raw_data,1,True)
raw_data = bbands(raw_data,1, True)
#raw_data = atr(raw_data,1, True, multiplier = 1.5)

raw_data = macd(raw_data,3,True)
raw_data = rsi(raw_data,4, True)
raw_data = aroon(raw_data,5, True)
obv(raw_data,2, True)
#pivotPoint(raw_data, pos=1, plot=False)
annotateBuySell(raw_data.iloc[[100,150]],"Buy")
#annotateBuySell(raw_data.iloc[[400,450]], "Sell")
#annotateBuySell(raw_data.iloc[[100,400, 550]],)

raw_data = detectCDPattern(raw_data,strPlot="engulfing, hammer", plot=True)

#iplot(fig, filename="analysis")


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [34]:
raw_data

,close,high,low,open,volume,bbup,bbmid,bblow,macd,macdsignal,macdhist,RSI,aroondown,aroonup,obv,text
date,,,,,,,,,,,,,,,,
2019-04-04 09:15:00,2018.10,2018.10,2006.30,2008.00,50107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50107.0,NaN
2019-04-04 09:16:00,2021.20,2021.65,2016.20,2018.20,56347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106454.0,NaN
2019-04-04 09:17:00,2020.50,2021.35,2018.60,2021.35,33322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139776.0,NaN
2019-04-04 09:18:00,2020.00,2020.90,2019.15,2019.50,15102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154878.0,NaN
2019-04-04 09:19:00,2020.80,2020.80,2018.65,2020.00,13210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168088.0,NaN
2019-04-04 09:20:00,2016.90,2021.60,2016.05,2021.60,12116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180204.0,NaN
2019-04-04 09:21:00,2019.90,2020.50,2016.95,2017.40,10715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190919.0,NaN
2019-04-04 09:22:00,2019.00,2020.00,2018.05,2019.90,8151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199070.0,NaN
2019-04-04 09:23:00,2018.70,2019.90,2017.60,2018.70,6773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205843.0,NaN


# Algorithm

# Visualization

In [25]:
iplot(fig, filename="analysis")

# WebSocket Handler

In [271]:
import multiprocessing

def worker():
    """worker function"""
    print 'Worker'
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker)
        jobs.append(p)
        p.start()

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int 'Worker')? (<ipython-input-271-be6822d7f749>, line 5)